In [0]:
# %sh
# pip install darts
# conda install -c conda-forge -c pytorch u8darts-all

In [0]:
import os
import pickle
import sys
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape, smape, mae
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression, GumbelLikelihood, GaussianLikelihood

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    TFTModel,
    LinearRegressionModel,
    LightGBMModel,
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)

from darts.metrics.metrics import mape, smape, ope


from torchmetrics import MeanAbsolutePercentageError
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# modeling packages
os.chdir('../..')

import mlflow
import src.modeling.darts_tft_wrapper as m
from src.modeling.darts_tft_wrapper import DartsTFTGlobalModel

In [0]:
import torch
torch.cuda.is_available()

In [0]:
print(torch.version.cuda)

In [0]:
# import src.data_engineering.data_engineering as de
# import src.feature_engineering.feature_engineering as fe

# Spark connection

In [0]:
if 'spark' not in locals():
    from databricks.connect import DatabricksSession
    from dotenv import load_dotenv
    load_dotenv()
    
    spark = DatabricksSession.builder.remote(
      host       = f"https://{os.environ['DATABRICKS_HOST']}",
      token      = os.environ['SPP_WEIS'],
      cluster_id = os.environ['CLUSTER_ID']
    ).getOrCreate()

# get data from spark

In [0]:
refresh_data = False

In [0]:
## lmp
if refresh_data:
    query = '''
    SELECT 
      GMTIntervalEnd_HE as GMTIntervalEnd,
      PNODE_Name,
      avg(LMP) as LMP_HOURLY
    FROM sandbox_data_science.spp_weis.lmp
    GROUP BY GMTIntervalEnd_HE, PNODE_Name
    '''
    res = spark.sql(query).collect()
    df = spark.createDataFrame(res).toPandas()
    df.to_parquet('lmp_df.parquet')
    display(df.head())


In [0]:
## mtrf
if refresh_data:
    query = 'SELECT * FROM sandbox_data_science.spp_weis.mtrf'
    res = spark.sql(query).collect()
    df = spark.createDataFrame(res).toPandas()
    df.to_parquet('mtrf_df.parquet')
    display(df.head())

In [0]:
## mtlf
if refresh_data:
    query = 'SELECT * FROM sandbox_data_science.spp_weis.mtlf'
    res = spark.sql(query).collect()
    df = spark.createDataFrame(res).toPandas()
    df.to_parquet('mtlf_df.parquet')
    display(df.head())

# Load dataframes

In [0]:
price_df = pd.read_parquet('lmp_df.parquet')
mtlf_df = pd.read_parquet('mtlf_df.parquet').sort_values('GMTIntervalEnd')
mtrf_df = pd.read_parquet('mtrf_df.parquet').sort_values('GMTIntervalEnd')

# Feature Engineering

In [0]:
########################################
# fill missing values
########################################
def fill_missed_values(df):
    """
    """
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')

    return df


########################################
# lmp
########################################
def get_psco_price_df(price_df):
    """
    """
    psco_idx = price_df["PNODE_Name"].str.contains("PSCO", case=False)
    psco_price_df = price_df[psco_idx]
    psco_price_df = psco_price_df.pivot_table(
                            index='GMTIntervalEnd',
                            columns='PNODE_Name',
                            values='LMP_HOURLY',
                            margins=False,
                        ).reset_index()

    psco_price_df.columns.name=None

    ls_nodes_name = list(psco_price_df.columns[1:])

    return fill_missed_values(psco_price_df), ls_nodes_name


def create_psco_price_series(psco_price_df, node_name_ls):
    """
    """
    psco_price_series = TimeSeries.from_dataframe(psco_price_df, time_col='GMTIntervalEnd', value_cols=node_name_ls, fill_missing_dates=True, freq='H', fillna_value=0, static_covariates=None, hierarchy=None).astype(np.float32)

    return psco_price_series


########################################
# mtlf
########################################
def create_mtlf_series(mtlf_df):
    """
    """
    mtlf_df = fill_missed_values(mtlf_df)
    mtlf_series = TimeSeries.from_dataframe(mtlf_df, time_col='GMTIntervalEnd', value_cols='MTLF', fill_missing_dates=True, freq='H', fillna_value=0, static_covariates=None, hierarchy=None).astype(np.float32)
    avg_act_series = TimeSeries.from_dataframe(mtlf_df, time_col='GMTIntervalEnd', value_cols='Averaged_Actual', fill_missing_dates=True, freq='H', fillna_value=0, static_covariates=None, hierarchy=None).astype(np.float32)
    
    return mtlf_series, avg_act_series
          

########################################
# mtrf
########################################  
# Add renewable/load ratio feature to mtrf dataframe
def add_enrgy_ratio_to_mtrf(mtlf_df, mtrf_df):
    """
    """
    mtrf_df = mtrf_df[['GMTIntervalEnd', 'Wind_Forecast_MW', 'Solar_Forecast_MW']].set_index('GMTIntervalEnd').asfreq('H').sort_index()
    mtrf_df = mtrf_df.join(mtlf_df[['GMTIntervalEnd','MTLF']].set_index('GMTIntervalEnd').asfreq('H').sort_index(), on='GMTIntervalEnd', how='outer').sort_values('GMTIntervalEnd').reset_index(drop=True)
    mtrf_df['Ratio'] = (mtrf_df['Wind_Forecast_MW'] + mtrf_df['Solar_Forecast_MW']) / mtrf_df['MTLF']
    mtrf_df.drop('MTLF', axis=1, inplace=True) 

    return fill_missed_values(mtrf_df)

def create_mtrf_series(mtrf_ratio_df):
    """
    """
    mtrf_series= TimeSeries.from_dataframe(mtrf_ratio_df, time_col='GMTIntervalEnd', value_cols=['Wind_Forecast_MW','Solar_Forecast_MW', 'Ratio'], fill_missing_dates=True, freq='H', fillna_value=0, static_covariates=None, hierarchy=None).astype(np.float32)
    
    return mtrf_series     

In [0]:
# lmp
psco_lmp_df, list_nodes_name = get_psco_price_df(price_df)
lmp_series = create_psco_price_series(psco_lmp_df, list_nodes_name)

# mtlf series
mtlf_series, avg_act_series = create_mtlf_series(mtlf_df)

# mtrf series
mtrf_ratio_df = add_enrgy_ratio_to_mtrf(mtlf_df, mtrf_df)
mtrf_series = create_mtrf_series(mtrf_ratio_df)

# Preprocess series

In [0]:

def lmp_series_drop_horizon(lmp_series, start_time, forecast_horizon):
    """
    """
    start_time_lmp = start_time
    end_time_lmp = lmp_series.end_time() - pd.Timedelta(f'{forecast_horizon+1}H')
    lmp_series = lmp_series.drop_before(start_time_lmp)
    lmp_series = lmp_series.drop_after(end_time_lmp)
    return lmp_series

def get_train_cutoff(lmp_series):
    """
    """
    #keep last 30 days (720 data points + horizon(72 hours)) of target series for validation. 
    training_cutoff = lmp_series[:-792].end_time()
    return training_cutoff

def get_lmp_train_test_series(lmp_series_drop_horizon, training_cutoff, forecast_horizon, input_chunk_length):
    """
    """
    lmp_series_train = lmp_series_drop_horizon.drop_after(training_cutoff - pd.Timedelta(f'{forecast_horizon+1}H')) # for future covariates
    lmp_series_val = lmp_series_drop_horizon.drop_before(training_cutoff + pd.Timedelta(f'{input_chunk_length+1}H'))
    lmp_series_all = lmp_series_drop_horizon

    return [lmp_series_train, lmp_series_val, lmp_series_all]


def scale_lmp_series(lmp_series_train, lmp_series_val, lmp_series_all):
    """
    """
    transformer_lmp = Scaler()
    lmp_series_train_transformed = transformer_lmp.fit_transform(lmp_series_train)
    lmp_series_val_transformed = transformer_lmp.transform(lmp_series_val)
    lmp_series_transformed = transformer_lmp.transform(lmp_series_all)

    return [lmp_series_train_transformed, lmp_series_val_transformed, lmp_series_transformed]

############## avg_act ##################
# def get_avg_act_train_test_series(avg_act_series, start_time, training_cutoff, forecast_horizon, input_chunk_length):
#     """
#     """
#     start_time_avg = start_time
#     end_time_avg = avg_act_series.end_time() - pd.Timedelta(f'{forecast_horizon+1}H')
#     avg_act_series = avg_act_series.drop_before(start_time_avg)
#     avg_act_series = avg_act_series.drop_after(end_time_avg)


#     avg_act_series_train = avg_act_series.drop_after(training_cutoff - pd.Timedelta(f'{forecast_horizon+1}H')) # for future covariates
#     avg_act_series_val = avg_act_series.drop_before(training_cutoff + pd.Timedelta(f'{input_chunk_length+1}H'))

#     return [avg_act_series_train, avg_act_series_val, avg_act_series]


# def scale_avg_act_series(avg_act_series_train, avg_act_series_val, avg_act_series):
#     """
#     """
#     transformer_avg_act = Scaler()
#     avg_act_series_train_transformed = transformer_avg_act.fit_transform(avg_act_series_train)
#     avg_act_series_val_transformed = transformer_avg_act.transform(avg_act_series_val)
#     avg_act_series_transformed = transformer_avg_act.transform(avg_act_series)

#     return [avg_act_series_train_transformed, avg_act_series_val_transformed, avg_act_series_transformed]

############## mtlf #####################
def get_mtlf_train_test_series(mtlf_series, start_time, training_cutoff):
    """
    """
    # drop times before the starting time
    mtlf_series = mtlf_series.drop_before(start_time)

    # Split
    mtlf_series_train = mtlf_series.drop_after(training_cutoff)
    mtlf_series_val = mtlf_series.drop_before(training_cutoff)

    return [mtlf_series_train, mtlf_series_val, mtlf_series]


def scale_mtlf_series(mtlf_series_train, mtlf_series_val, mtlf_series):
    """
    """
    transformer_mtlf = Scaler()
    mtlf_series_train_transformed = transformer_mtlf.fit_transform(mtlf_series_train)
    mtlf_series_val_transformed = transformer_mtlf.transform(mtlf_series_val)
    mtlf_series_transformed = transformer_mtlf.transform(mtlf_series)

    return [mtlf_series_train_transformed, mtlf_series_val_transformed, mtlf_series_transformed]

############# avg_actual #################
def get_avg_act_train_test_series(avg_act_series, start_time, training_cutoff):
    """
    """
    avg_act_series = avg_act_series.drop_before(start_time)

    # Split
    avg_act_series_train = avg_act_series.drop_after(training_cutoff)
    avg_act_series_val = avg_act_series.drop_before(training_cutoff)

    return [avg_act_series_train, avg_act_series_val, avg_act_series]


def scale_avg_act_series(avg_act_series_train, avg_act_series_val, avg_act_series):
    """
    """
    transformer_avg = Scaler()
    avg_act_series_train_transformed = transformer_avg.fit_transform(avg_act_series_train)
    avg_act_series_val_transformed = transformer_avg.transform(avg_act_series_val)
    avg_act_series_transformed = transformer_avg.transform(avg_act_series)

    return [avg_act_series_train_transformed, avg_act_series_val_transformed, avg_act_series_transformed]


############## mtrf #####################
def get_mtrf_train_test_series(mtrf_series, start_time, training_cutoff):
    """
    """
    mtrf_series = mtrf_series.drop_before(start_time)

    # Split
    mtrf_series_train = mtrf_series.drop_after(training_cutoff)
    mtrf_series_val = mtrf_series.drop_before(training_cutoff)

    return [mtrf_series_train, mtrf_series_val, mtrf_series]


def scale_mtrf_series(mtrf_series_train, mtrf_series_val, mtrf_series):
    """
    """
    transformer_win = Scaler()
    wind_series_train_transformed = transformer_win.fit_transform(mtrf_series_train)
    wind_series_val_transformed = transformer_win.transform(mtrf_series_val)
    wind_series_transformed = transformer_win.transform(mtrf_series)

    return [wind_series_train_transformed, wind_series_val_transformed, wind_series_transformed]




In [0]:
start_time = pd.Timestamp('2023-04-02 00:00:00')
### TIME CHANGE ########################################################
input_chunk_length = 24*7
forecast_horizon = 24*3
training_cutoff = pd.Timestamp("2023-06-01 06:00:00")
########################################################################
lmp_series = lmp_series_drop_horizon(lmp_series, start_time, forecast_horizon)
# training_cutoff = get_train_cutoff(lmp_series_drop_horizon)
lmp_series_train, lmp_series_val, lmp_series_all = get_lmp_train_test_series(lmp_series, training_cutoff, forecast_horizon, input_chunk_length)
lmp_series_train_transformed, lmp_series_val_transformed, lmp_series_transformed = scale_lmp_series(lmp_series_train, lmp_series_val, lmp_series_all)

print(f'train start: {lmp_series_train.start_time()}')
print(f'train end: {lmp_series_train.end_time()}')
print(f'val start: {lmp_series_val.start_time()}')
print(f'val end: {lmp_series_val.end_time()}')


mtlf_series_train, mtlf_series_val, mtlf_series = get_mtlf_train_test_series(mtlf_series, start_time, training_cutoff)
mtlf_series_train_transformed, mtlf_series_val_transformed, mtlf_series_transformed = scale_mtlf_series(mtlf_series_train, mtlf_series_val, mtlf_series)

print(f'train start: {mtlf_series_train.start_time()}')
print(f'train end: {mtlf_series_train.end_time()}')
print(f'val start: {mtlf_series_val.start_time()}')
print(f'val end: {mtlf_series_val.end_time()}')


avg_act_series_train, avg_act_series_val, avg_act_series = get_avg_act_train_test_series(avg_act_series, start_time, training_cutoff)
avg_act_series_train_transformed, avg_act_series_val_transformed, avg_act_series_transformed = scale_avg_act_series(avg_act_series_train, avg_act_series_val, avg_act_series)

print(f'train start: {avg_act_series_train.start_time()}')
print(f'train end: {avg_act_series_train.end_time()}')
print(f'val start: {avg_act_series_val.start_time()}')
print(f'val end: {avg_act_series_val.end_time()}')



mtrf_series_train, mtrf_series_val, mtrf_series = get_mtrf_train_test_series(mtrf_series, start_time, training_cutoff)
mtrf_series_train_transformed, mtrf_series_val_transformed, mtrf_series_transformed = scale_mtrf_series(mtrf_series_train, mtrf_series_val, mtrf_series)

print(f'train start: {mtrf_series_train.start_time()}')
print(f'train end: {mtrf_series_train.end_time()}')
print(f'val start: {mtrf_series_val.start_time()}')
print(f'val end: {mtrf_series_val.end_time()}')

In [0]:
past_cov_train = avg_act_series_train_transformed
past_cov_val = avg_act_series_val_transformed
past_cov = avg_act_series_transformed

In [0]:
# Concatenate future training covariates
future_covariates_train = concatenate([mtlf_series_train_transformed, mtrf_series_train_transformed], axis=1)
future_covariates_train.values().shape

In [0]:
# Concatenate future validation covariates
end_time = mtlf_series_val_transformed.end_time() + pd.Timedelta('1H')
mtrf_series_val_transformed_end_droped = mtrf_series_val_transformed.drop_after(end_time)

future_covariates_val = concatenate([mtlf_series_val_transformed, mtrf_series_val_transformed_end_droped], axis=1)
future_covariates_val.values().shape

In [0]:
# Concatenate the entire covariate series
mtrf_series_transformed_end_droped = mtrf_series_transformed.drop_after(end_time)

future_covariates = concatenate([mtlf_series_transformed, mtrf_series_transformed_end_droped], axis=1)
future_covariates.values().shape

In [0]:
lmp_train_all = []
for i in range(len(list_nodes_name)):
    lmp_train_all.append(lmp_series_train_transformed[list_nodes_name[i]])


lmp_val_all = []
for i in range(len(list_nodes_name)):
    lmp_val_all.append(lmp_series_val_transformed[list_nodes_name[i]])

# Modeling

## Create TFT model

**Considerations**
- Target series: `LMP`
- Past Covariate series: `Averaged_actula` 
- Future Covariate Series: `MTLF`, `Wind`, and `Solar`. Basically, these are mid-term load forecasts and mid-term renewables forecasts.
- `input_chunk-length = 168 (7 days)`, `output_chunk_length = 24 hours` and **forecasting horizon could be the length of output_chunk or my validation set?**

##### Note : You can pass the covariates to the model as individual series in a list, or you can stack them and pass them as one multivariate series. However, you cannot sack them if they do not have the same length.
##### Note from Dart's TFT example: we can just provide the ***whole covariates series*** as future_covariates argument to the model; the model will slice these covariates and use only what it needs in order to train on forecasting the target train_transformed.

## Fit the model using covariate series

In [0]:
# Parameters
num_samples = 200

figsize = (16, 5)
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

In [0]:
# default quantiles for QuantileRegression
quantiles = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.99,
]

encoders = {"cyclic": {"future": ["hour", "dayofweek"]},
            "transformer": Scaler()}

In [0]:
torch_metrics = MeanAbsolutePercentageError()

my_stopper = EarlyStopping(
    monitor= "val_MeanAbsolutePercentageError", # "val_loss",
    patience=5,
    min_delta=0.05,
    mode='min',
)

pl_trainer_kwargs={"callbacks": [my_stopper],
                   "accelerator": "gpu", "devices": [0]}

# Log Darts TFT wrapper model

In [0]:
# https://stackoverflow.com/questions/68356746/changing-subdirectory-of-mlflow-artifact-store
artifact_path = "./model_artifacts/"
os.makedirs(artifact_path, exist_ok=True)

try:
    experiment = mlflow.get_experiment_by_name('tft')
    experiment_id = experiment.experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment('tft', artifact_location=artifact_path)

In [0]:
experiment_id

In [0]:
# !pip list

In [0]:
# !pip install lightning

In [0]:
# turn off pytorch lightning logs
# https://github.com/Lightning-AI/lightning/issues/3431
# import logging
# import lightning
# logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.ERROR)
# logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.ERROR)
# # logging.getLogger("lightning.pytorch.accelerators").setLevel(logging.WARNING)
# logging.getLogger("lightning").setLevel(logging.ERROR)

In [0]:
run_backtest = True

with mlflow.start_run(experiment_id=experiment_id) as run:
    print(f'run: {run}')

    lr = 1e-4
    hidden_size = 32
    lstm_layers = 1
    num_attention_heads = 4
    
    tft_model = TFTModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=forecast_horizon,
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=0.1,
        batch_size=16,
        n_epochs=2,
        add_encoders=encoders,
        likelihood=QuantileRegression(quantiles=quantiles),  # QuantileRegression is set per default
        optimizer_kwargs={"lr": lr},
        random_state=42,
        torch_metrics=torch_metrics,
        pl_trainer_kwargs=pl_trainer_kwargs,
    )
    
    tft_model.fit(
        series=[lmp_train_all[0], lmp_train_all[1]],
        val_series=[lmp_val_all[0], lmp_val_all[1]],
        past_covariates=[past_cov_train for i in range(2)],
        val_past_covariates=[past_cov_val for i in range(2)],
        future_covariates=[future_covariates_train for i in range(2)],
        val_future_covariates=[future_covariates_val for i in range(2)],
        verbose=True)


    # log parameters for the run
    # need to add accuracy results here...
    params = {
        "lr":lr,
        "epochs_trained":tft_model.epochs_trained,
        "num_attention_heads":tft_model.num_attention_heads,
        "hidden_size":tft_model.hidden_size,
        "lstm_layers":tft_model.lstm_layers,
        }

    # backtesting takes a moment and generates a lot of output
    # we can turn it off for testing
    if run_backtest:
        # back test on validation data
        acc = tft_model.backtest(
            series=[lmp_val_all[0], lmp_val_all[1]],
            past_covariates=[past_cov_val for i in range(2)],
            future_covariates=[future_covariates_val for i in range(2)],
            retrain=False,
            stride=6,
            metric=[mape, smape, ope],
            verbose=False,
        )
        
        acc_df = pd.DataFrame(
            np.mean(acc, axis=0).reshape(1,-1),
            columns=['mape', 'smape', 'ope']
        )

        # add accuracy to params
        params['mape'] = acc_df.mape[0]
        params['smape'] = acc_df.smape[0]
        params['ope'] = acc_df.ope[0]
    

    # set up path to save model
    model_name = "tft_model"
    model_path = ''.join([artifact_path, model_name])

    # log params
    params['model_name'] = model_name
    mlflow.log_params(params)

    # save tft model files (tft_model, tft_model.ckpt) 
    # and load them to artifacts when logging the model
    tft_model.save(model_path)

    # map model artififacts in dictionary
    artifacts = {
        'model': model_path,
        'model.ckpt': model_path+'.ckpt'
    }

    # log model
    mlflow.pyfunc.log_model(
        artifact_path='model_artifacts',
        artifacts=artifacts,
        # model will get loaded from artifacts, we don't need instantiate with one
        python_model=DartsTFTGlobalModel(), 
    ) # registered_model_name="summarization-model"
  

# Load logged model and make prediction

In [0]:
node_series = lmp_series_transformed[list_nodes_name[0]]
past_cov_series = avg_act_series_transformed
future_cov_series = future_covariates

data = {
    'series': [node_series],
    'past_covariates': [past_cov_series],
    'future_covariates': [future_cov_series],
    'n': forecast_horizon,
    'num_samples': num_samples
}

# Predict on a Pandas DataFrame.
import pandas as pd
model_input = pd.DataFrame(data)

In [0]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in os.listdir(artifact_path) if isfile(join(artifact_path, f))]

In [0]:
# get paths to logged models
model_files = []
for root, dirs, files in os.walk(artifact_path):
    for f in files:
        path = os.path.join(root, f)
        if 'model_artifacts/artifacts/tft_model' in path and '.ckpt' not in path:
            model_files += [path]

model_files

In [0]:
# Load model as a PyFuncModel.
path = model_files[0]
print(path)
# need the directory with 'MLmodel'
path = '/'.join(path.split('/')[:-2])
print(path)

In [0]:
loaded_model =  mlflow.pyfunc.load_model(path)
type(loaded_model)

In [0]:
pred_series = loaded_model.predict(model_input)

In [0]:
pred_series.plot()